In [16]:
import pandas as pd
import  pathlib


In [17]:
#type of cancer to select from table with chromothripsis data
cancer_type = "Prost-AdenoCA"

#path to folder with SVs
sv_path = 'data/SVs/'

#path to file with metadata
metadata_path = "data/general data/release.csv"
#or download data and choose one of lists
#!wget https://dcc.icgc.org/api/v1/download?fn=/PCAWG/donors_and_biospecimens/pcawg-data-releases.xlsx -O release.xlsx --no-check-certificate

#path to file with chromothripsis data
chromo_path = "data/general data/stat_chromothripsis.csv"

In [18]:
#name of column in metadata table with IDs matching SVs file names
meta_id_column = "tumor_wgs_submitter_sample_id"

#names of columns in SVs tables
start1 = "start1"
start2 = "start2"
end1 = "end1"
end2 = "end2"
chrom1 = "chrom1"
chrom2 = "chrom2"
sv_id = "sv_id"
pe_support = "pe_support"
strand1 = "strand1"
strand2 = "strand2"
svclass = "svclass"
svmethod = "svmethod"

#name of column with chromothripsis determination result in table with chromothripsis
chromo_label = "chromo_label"

In [19]:
#Load table with metadata
donors = pd.read_csv(metadata_path,delimiter=';')

#Load table with chromothripsis data
stat_chromo = pd.read_csv(chromo_path,delimiter=',')

#Load names of fies with SVs
SVs_names = []
for currentFile in pathlib.Path(sv_path).iterdir():
    SVs_names.append(currentFile.name.replace(".csv","").__str__())

D:\Programs\Anaconda\envs\scientific\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (53) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [20]:
#future columns of table
unique_id_series = []
sv_file_id_series = []
chrom_1_series = []
start1_series=[]
chrom2_series = []
start2_series = []
sv_id_series = []
pe_support_series = []
strand1_series = []
strand2_series = []
sv_class_series = []
sv_method_series = []
chromo_label_1_series = []
chromo_label_2_series = []


for sv_id_file in SVs_names:
    search_id_row=donors.loc[donors[meta_id_column]==sv_id_file]
    if (search_id_row.empty):
        continue
    donor_unique_id = search_id_row.donor_unique_id.iloc[0]
    sv_table = pd.read_csv(sv_path+sv_id_file+".csv",delimiter='\t')
    chromo_data_for_sv = stat_chromo[stat_chromo.donor_unique_id==donor_unique_id][stat_chromo.histo==cancer_type]
    if(chromo_data_for_sv.empty):
        continue
    for index,row in sv_table.iterrows():
        #validate SV position
        assert row[start1]+1==row[end1]
        assert row[start2]+1==row[end2]

        unique_id_series.append(donor_unique_id)
        sv_file_id_series.append(sv_id_file)
        chrom_1_series.append(row[chrom1])
        start1_series.append(row[start1])
        chrom2_series.append(row[chrom2])
        start2_series.append(row[start2])
        sv_id_series.append(row[sv_id])
        pe_support_series.append(row[pe_support])
        strand1_series.append(row[strand1])
        strand2_series.append(row[strand2])
        sv_class_series.append(row[svclass])
        sv_method_series.append(row[svmethod])

        #replace Y chromosome name with X
        if(row[chrom1]=="Y"):
            row[chrom1]="X"
        if(row[chrom2]=="Y"):
            row[chrom2]="X"

        chromo_label_1_series.append(chromo_data_for_sv[chromo_data_for_sv.Chr==row[chrom1].__str__()][[chromo_label]].iloc[0].iloc[0])
        chromo_label_2_series.append(chromo_data_for_sv[chromo_data_for_sv.Chr==row[chrom2].__str__()][[chromo_label]].iloc[0].iloc[0])
result_table = pd.DataFrame({ 'unique_id': unique_id_series,
                              'sv_file_id': sv_file_id_series,
                              'chrom1': chrom_1_series,
                              'start1': start1_series,
                              'chrom2': chrom2_series,
                              'start2': start2_series,
                              'sv_id': sv_id_series,
                              'pe_support': pe_support_series,
                              'strand1': strand1_series,
                              'strand2': strand2_series,
                              'svclass': sv_class_series,
                              'svmethod': sv_method_series,
                              'chromo_label1': chromo_label_1_series,
                              'chromo_label2': chromo_label_2_series})
result_table.to_csv("result.csv",sep="\t")






C:\Users\vit\AppData\Local\Temp/ipykernel_11324/2390061413.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chromo_data_for_sv = stat_chromo[stat_chromo.donor_unique_id==donor_unique_id][stat_chromo.histo==cancer_type]
